# 계류 법안 분석 

작성자: 박하람

In [1]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [3]:
bill_df = pd.read_csv('bill_20th_data_final.csv', encoding='utf-8', parse_dates=['제안일자','의결일자'])
bill_df = bill_df[['의안번호','의안명','제안자구분','제안일자','의결일자','의결결과','제안회기','제안이유','소관위원회','법률반영여부']]

In [4]:
bill_df.shape

(23684, 10)

In [5]:
bill_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23684 entries, 0 to 23683
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   의안번호    23684 non-null  int64         
 1   의안명     23684 non-null  object        
 2   제안자구분   23684 non-null  object        
 3   제안일자    23684 non-null  datetime64[ns]
 4   의결일자    23684 non-null  datetime64[ns]
 5   의결결과    23684 non-null  object        
 6   제안회기    23684 non-null  object        
 7   제안이유    23016 non-null  object        
 8   소관위원회   23454 non-null  object        
 9   법률반영여부  23684 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 1.8+ MB


In [6]:
# NaN 채우기 
bill_df = bill_df.fillna('')
bill_df.isnull().sum()

의안번호      0
의안명       0
제안자구분     0
제안일자      0
의결일자      0
의결결과      0
제안회기      0
제안이유      0
소관위원회     0
법률반영여부    0
dtype: int64

## 데이터 전처리 

- 법률안만 선정

> 의안의 결과를 예측하기 위해서는 먼저 문제의 특성을 잘 이해하는 것이 중요하다. 먼저 의안에는 기존 법률을 수정, 폐지하거나 새로운 법률을 제안하는 법률안만 있는 것이 아니라 결의안, 동의안, 출석요구안, 예산안, 정부 관리직의 임명선출안 등도 포함된다. 그 중에서 일반 시민 및 각종 이익 단체에 영향을 가장 크게 주는 것은 법을 수정하는 법률안이므로, 이 연구에서는 법률안에 중점을 둔다.(선행연구)

#### 법률안 추리기

In [7]:
def split_name(x): 
    name = x.split('(')[0]
    return name

def preprocessing(text):
    # 특수문자 제거
    text = re.sub('[?.,;:|\)(*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

In [8]:
bill_df['의안명'] = bill_df['의안명'].apply(lambda x: split_name(x))

In [9]:
bill_df.loc[bill_df['의안명'].str.endswith('법률안') == True, '법률안'] = 1
bill_df.loc[bill_df['의안명'].str.endswith('법률안') == False, '법률안'] = 0

In [10]:
bill_df['법률안'].value_counts()

1.0    22453
0.0     1231
Name: 법률안, dtype: int64

In [11]:
bill_df = bill_df[bill_df['법률안'] == 1]

In [12]:
len(bill_df)

22453

#### 의안명, 제안이유 클렌징

In [13]:
%time bill_df['의안명'] = bill_df['의안명'].apply(lambda x: preprocessing(x))
%time bill_df['제안이유'] = bill_df['제안이유'].apply(lambda x: preprocessing(x))

CPU times: user 91.1 ms, sys: 1.7 ms, total: 92.8 ms
Wall time: 93.8 ms
CPU times: user 1.23 s, sys: 13.5 ms, total: 1.25 s
Wall time: 1.25 s


## EDA 

### 법률 반영/미반영 기준으로 의안처리시 소요시간 분석

- 법률반영: 원안가결, 원안수정, 대안반영, 수정안반영
- 법률미반영: 부결, 폐기, 철회, 반려, 기타 

In [14]:
bill_df['의결결과'].unique()

array(['임기만료폐기', '원안가결', '수정가결', '대안반영폐기', '철회', '폐기', '부결', '수정안반영폐기'],
      dtype=object)

- accept_list = ['대안반영폐기', '원안가결', '수정가결', '수정안반영폐기', '가결']
- reject_list = ['임기만료폐기', '부결', '철회', '반려', '심사대상제외', '폐기']

In [15]:
bill_tmp = bill_df['의결결과'].value_counts()
bill_tmp

임기만료폐기     14269
대안반영폐기      5003
원안가결        1964
수정가결         852
철회           201
폐기           121
수정안반영폐기       41
부결             2
Name: 의결결과, dtype: int64

In [16]:
bill_df['법률반영여부'].value_counts()

미반영    14593
반영      7860
Name: 법률반영여부, dtype: int64

In [17]:
bill_df['의결처리기간'] = (bill_df['의결일자'] - bill_df['제안일자']).dt.days
bill_df.head(3)

,의안번호,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,법률반영여부,법률안,의결처리기간
0,2024996,집합건물의 소유 및 관리에 관한 법률 일부개정법률안,의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,아파트 등 공동주택은 공동주택관리법 에 의해 체계적으로 관리되고 있는 반면 ...,법제사법위원회,미반영,1.0,7
1,2024995,지방세법 일부개정법률안,의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,현행법상 아파트 등 주택에 대한 재산세 과세표준은 국토교통부가 해마다 부동산 가격...,행정안전위원회,미반영,1.0,7
2,2024994,법률용어 정비를 위한 정보위원회 소관 개 법률 일부개정을 위한 법률안,위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,정보위원회,반영,1.0,0


In [18]:
bill_df['의결처리기간'].groupby(bill_df['법률반영여부']).mean()

법률반영여부
미반영    759.561228
반영     253.647837
Name: 의결처리기간, dtype: float64

In [19]:
pend_df = pd.pivot_table(bill_df, 
                        index=['법률반영여부', '의결결과'],
                        values=['의결처리기간']).fillna(0)
pend_df

의결처리기간
법률반영여부 의결결과               
미반영    부결       217.500000
       임기만료폐기   772.238419
       철회        83.865672
       폐기       395.991736
반영     대안반영폐기   321.562063
       수정가결     283.303991
       수정안반영폐기  207.975610
       원안가결      68.734725

In [20]:
# 의결처리기간 - 평균 
pd.merge(pend_df, bill_tmp, left_on='의결결과', right_index=True)

의결처리기간   의결결과
법률반영여부 의결결과                      
미반영    부결       217.500000      2
       임기만료폐기   772.238419  14269
       철회        83.865672    201
       폐기       395.991736    121
반영     대안반영폐기   321.562063   5003
       수정가결     283.303991    852
       수정안반영폐기  207.975610     41
       원안가결      68.734725   1964

In [21]:
print('미반영 평균 소요시간: ', pend_df.loc['미반영','의결처리기간'].mean())
print('반영 평균 소요시간: ', pend_df.loc['반영','의결처리기간'].mean())

미반영 평균 소요시간:  367.3989565322882
반영 평균 소요시간:  220.39409704492132


#### 소관위원회별 법률반영여부 분석

법률반영여부에 대한 기준은 의안정보시스템에 따른 것. 그러나 대안반영폐기, 수정안반영폐기는 법률안이 되지 못하므로 이것이 과연 법률이 반영 되었다 볼 수 있는가의 문제도 존재. 

> 일반적으로 법률안들은 임기만료폐기, 대안반영폐기, 폐기, 철회, 부결, 공포 등 6가지 상태 중 하나로 끝나게 된다. 그 중에서 공포된 법률안만이 성공적으로 “통과”되었다고 볼 수 있으며, 공포 단계에 이르기 전에 현재 국회의 임기가 만료되거나(임기만료폐기), 같은 목적을 가진 다른 대안이 반영되어 원안이 폐 기되거나(대안반영폐기), 발의한 의원들이 철회를 하는 경우 법률안은 법이 되지 못한다. 본 연구에서는 법률안의 발의 시점, 즉 접수 단계에서 해당 법률안이 공포될 것인지의 여부를 예측하는 것을 목표로 한다. (선행연구) 

### 법률공포여부 분석

In [22]:
comm_df = bill_df.copy()

In [23]:
proclaim_list = ['원안가결', '수정가결']
reject_list = ['임기만료폐기', '대안반영폐기', '철회', '폐기', '부결', '수정안반영폐기']

In [24]:
comm_df.loc[comm_df['의결결과'].isin(proclaim_list), '법률공포여부'] = '공포'
comm_df.loc[comm_df['의결결과'].isin(reject_list), '법률공포여부'] = '미공포'

In [25]:
# 기타 index 번호: 9443, 10301, 10508 (소관위원회 없는 경우)
comm_df[comm_df['소관위원회'] == '']
comm_df.drop([9443, 10301, 10508],inplace=True)

In [26]:
comm_df['법률공포여부'].value_counts()

미공포    19634
공포      2816
Name: 법률공포여부, dtype: int64

#### 소관위원회별 법률공포여부 분석

국회법 제4장 회의 제2절 발의, 동의, 철회와 번안 

> 제33조 3번 의안이 발의 또는 제출되었을 때에는 의장은 이것을 국회에 보고한 후 적당한 위원회에 부탁하고 그 심사가 끝난 뒤에 본회의에 부의한다. **단, 법률안 이외의 의안은 국회의 결의에 의하여 위원회의 심사를 생략할 수 있다.**

**적어도 100개 이상**의 데이터가 있는 것으로 분석해야

In [27]:
comm = comm_df['소관위원회'].unique()
special_comm = [com for com in comm if com.endswith('특별위원회') == True]
basic_comm = [com for com in comm if com.endswith('특별위원회') == False]

print('상설 특별위원회 리스트:', '\n', special_comm, '\n')
print('상임위원회 리스트:', '\n', basic_comm)

상설 특별위원회 리스트: 
 ['헌법개정 및 정치개혁 특별위원회', '정치개혁 특별위원회', '사법개혁 특별위원회'] 

상임위원회 리스트: 
 ['법제사법위원회', '행정안전위원회', '정보위원회', '여성가족위원회', '문화체육관광위원회', '국회운영위원회', '과학기술정보방송통신위원회', '기획재정위원회', '국토교통위원회', '환경노동위원회', '외교통일위원회', '농림축산식품해양수산위원회', '정무위원회', '국방위원회', '교육위원회', '보건복지위원회', '산업통상자원중소벤처기업위원회', '교육문화체육관광위원회', '안전행정위원회', '산업통상자원위원회', '미래창조과학방송통신위원회']


In [28]:
comm_df.loc[comm_df['소관위원회'].isin(basic_comm), '위원회 분류'] = '상임위원회'
comm_df.loc[comm_df['소관위원회'].isin(special_comm), '위원회 분류'] = '특별위원회'

#### 상임위원회만 분석

In [29]:
comm_df1 = comm_df[comm_df['위원회 분류'] == '상임위원회']
comm_df1.head()

,의안번호,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,법률반영여부,법률안,의결처리기간,법률공포여부,위원회 분류
0,2024996,집합건물의 소유 및 관리에 관한 법률 일부개정법률안,의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,아파트 등 공동주택은 공동주택관리법 에 의해 체계적으로 관리되고 있는 반면 ...,법제사법위원회,미반영,1.0,7,미공포,상임위원회
1,2024995,지방세법 일부개정법률안,의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,현행법상 아파트 등 주택에 대한 재산세 과세표준은 국토교통부가 해마다 부동산 가격...,행정안전위원회,미반영,1.0,7,미공포,상임위원회
2,2024994,법률용어 정비를 위한 정보위원회 소관 개 법률 일부개정을 위한 법률안,위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,정보위원회,반영,1.0,0,공포,상임위원회
3,2024993,진실 화해를 위한 과거사정리 기본법 일부개정법률안,위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,행정안전위원회,반영,1.0,0,공포,상임위원회
4,2024992,화재예방 소방시설 설치 유지 및 안전관리에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,행정안전위원회,반영,1.0,0,공포,상임위원회


In [30]:
comm_df2 = pd.DataFrame(comm_df1.groupby('소관위원회')['법률공포여부'].value_counts()).unstack().fillna(0)
comm_df2.reset_index(inplace=True)
comm_df2.columns = ['소관위원회','공포','미공포']
comm_df2.head()

,소관위원회,공포,미공포
0,과학기술정보방송통신위원회,111,855
1,교육문화체육관광위원회,109,124
2,교육위원회,91,796
3,국방위원회,58,424
4,국토교통위원회,385,1751


In [31]:
comm_df2['합계'] = comm_df2['공포'] + comm_df2['미공포']
comm_df2['공포율'] = np.round((comm_df2['공포'] / comm_df2['합계']) * 100)

# 9, 12번은 의안수가 100이하라는 것을 감안하면, 아래 100 이상 의안수는 의미 있다. 왜 이렇게 나올까? 
comm_df2.sort_values(by='공포율', ascending=False)

,소관위원회,공포,미공포,합계,공포율
9,미래창조과학방송통신위원회,7,4,11,64.0
12,산업통상자원위원회,58,38,96,60.0
1,교육문화체육관광위원회,109,124,233,47.0
7,농림축산식품해양수산위원회,446,1158,1604,28.0
14,안전행정위원회,33,109,142,23.0
4,국토교통위원회,385,1751,2136,18.0
15,여성가족위원회,65,325,390,17.0
13,산업통상자원중소벤처기업위원회,199,998,1197,17.0
11,보건복지위원회,314,2158,2472,13.0
3,국방위원회,58,424,482,12.0


In [32]:
# 국회의원들의 관심도를 드러내는 것이 맞을까? 이렇게 많이 발의했다면? 
comm_df2.sort_values(by='합계', ascending=False)

,소관위원회,공포,미공포,합계,공포율
19,행정안전위원회,162,2502,2664,6.0
11,보건복지위원회,314,2158,2472,13.0
4,국토교통위원회,385,1751,2136,18.0
6,기획재정위원회,114,1902,2016,6.0
20,환경노동위원회,177,1791,1968,9.0
10,법제사법위원회,151,1667,1818,8.0
7,농림축산식품해양수산위원회,446,1158,1604,28.0
17,정무위원회,184,1385,1569,12.0
13,산업통상자원중소벤처기업위원회,199,998,1197,17.0
0,과학기술정보방송통신위원회,111,855,966,11.0


## 소관위원회별 토픽 모델링 



In [33]:
# !apt-get update 
# !apt-get install g++ openjdk-8-jdk python-dev python3-dev 
# !pip3 install JPype1-py3 
# !pip3 install konlpy 
# !JAVA_HOME="C:\Program Files\Java\jdk-13.0.2"
# !pip3 install nltk

zsh:1: command not found: apt-get
zsh:1: command not found: apt-get


In [34]:
# 한글폰트 설정
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

Unable to locate an executable at "/Library/Java/JavaVirtualMachines/jdk-14.0.1.jdk/Contents/Home/bin/apt" (-1)


In [35]:
comm_df.loc[comm_df['소관위원회'].isin(basic_comm), '위원회 분류'] = '상임위원회'
comm_df.loc[comm_df['소관위원회'].isin(special_comm), '위원회 분류'] = '특별위원회'

### 상임위원회 분석

In [36]:
comm_df1 = comm_df[comm_df['위원회 분류'] == '상임위원회']
comm_df1.head(3)

,의안번호,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,법률반영여부,법률안,의결처리기간,법률공포여부,위원회 분류
0,2024996,집합건물의 소유 및 관리에 관한 법률 일부개정법률안,의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,아파트 등 공동주택은 공동주택관리법 에 의해 체계적으로 관리되고 있는 반면 ...,법제사법위원회,미반영,1.0,7,미공포,상임위원회
1,2024995,지방세법 일부개정법률안,의원,2020-05-22,2020-05-29,임기만료폐기,제20대 (2016~2020) 제378회,현행법상 아파트 등 주택에 대한 재산세 과세표준은 국토교통부가 해마다 부동산 가격...,행정안전위원회,미반영,1.0,7,미공포,상임위원회
2,2024994,법률용어 정비를 위한 정보위원회 소관 개 법률 일부개정을 위한 법률안,위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,정보위원회,반영,1.0,0,공포,상임위원회


In [37]:
comm_df1['소관위원회'].value_counts()

행정안전위원회            2664
보건복지위원회            2472
국토교통위원회            2136
기획재정위원회            2016
환경노동위원회            1968
법제사법위원회            1818
농림축산식품해양수산위원회      1604
정무위원회              1569
산업통상자원중소벤처기업위원회    1197
과학기술정보방송통신위원회       966
교육위원회               887
문화체육관광위원회           745
국회운영위원회             515
국방위원회               482
여성가족위원회             390
외교통일위원회             284
교육문화체육관광위원회         233
안전행정위원회             142
산업통상자원위원회            96
정보위원회                31
미래창조과학방송통신위원회        11
Name: 소관위원회, dtype: int64

In [38]:
comm_df1['의안명+제안이유'] = (comm_df1['의안명'] + ' ' + comm_df1['제안이유']).str.strip()

<ipython-input-38-4d774aa0361d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comm_df1['의안명+제안이유'] = (comm_df1['의안명'] + ' ' + comm_df1['제안이유']).str.strip()


In [39]:
comm_anal1 = comm_df1[['소관위원회','의결결과','법률공포여부','의결처리기간','의안명+제안이유']]

for index, com in enumerate(basic_comm): 
    print(index, com)

0 법제사법위원회
1 행정안전위원회
2 정보위원회
3 여성가족위원회
4 문화체육관광위원회
5 국회운영위원회
6 과학기술정보방송통신위원회
7 기획재정위원회
8 국토교통위원회
9 환경노동위원회
10 외교통일위원회
11 농림축산식품해양수산위원회
12 정무위원회
13 국방위원회
14 교육위원회
15 보건복지위원회
16 산업통상자원중소벤처기업위원회
17 교육문화체육관광위원회
18 안전행정위원회
19 산업통상자원위원회
20 미래창조과학방송통신위원회


#### [0] 법제사법위원회

법제사법위원회(法制司法委員會)는 법제·사법에 관한 국회의 의사결정기능을 실질적으로 수행하는 국회 상임위원회이다.

In [40]:
com1 = comm_anal1[comm_anal1['소관위원회'] == basic_comm[0]]
com1.shape

(1818, 5)

In [41]:
com1['법률공포여부'].value_counts()

미공포    1667
공포      151
Name: 법률공포여부, dtype: int64

In [42]:
# 공포/미공포 분류
com1_pro = com1[com1['법률공포여부'] == '공포'].reset_index()
com1_unpro = com1[com1['법률공포여부'] == '미공포'].reset_index()

In [43]:
com1_pro.head(3)

,index,소관위원회,의결결과,법률공포여부,의결처리기간,의안명+제안이유
0,59,법제사법위원회,원안가결,공포,0,군사법원법 일부개정법률안 군인 군무원 등 사이에 발생한 범죄의 피해자에게 변호사가 ...
1,61,법제사법위원회,원안가결,공포,0,출입국관리법 일부개정법률안 가 출입국 전 과정에서 지문 얼굴을 비롯하여 홍채 ...
2,62,법제사법위원회,원안가결,공포,0,주택임대차보호법 일부개정법률안 현행법은 임대인은 임대차기간 종료 개월 전부터 개월 ...


In [44]:
import nltk 
from konlpy.tag import Okt

stopwords = ['을','이','의','를','에','가','들','은','는','으로','한','도',
              '수','에서','로','것','그','과','제','입니다','할','하고','적','하는',
              '합니다','와','에게','고','인','하여','등','저','있습니다','말','까지',
              '그리고','다','만','했습니다','안','된','못','일','더','위',
              '있는','해','또한','명','하지','정말','물','많은','것임',
              '중','게','너무','자','이런','때','되지','위해','에는','없는',
              '및','요','때문','관련','대한','전','라고','되었습니다','하기','나','후','같은','해서',
              '글','되어','주','차','하','내','없이','대해','잘','항','조','경우','함안','기간','하려는','일부',
             '마련','있도록','또는','함','하도록','개월','부터','제호','관','되고','년','월']

# 토큰화 해주기 
def tokenize(textlist): 
    okt = Okt()
    nounlist = []
    for i in range(len(textlist)):
        n = re.sub('[-=.#/:$}·,■?]','',textlist[i])
        k = re.sub('[0-9]','',n)
        nounlist.append(kkma.nouns(k))
    return nounlist 

# 의미없는 단어 삭제 
def rem_useless(nounlist, stopwords): 
    cleanlist = []
    removedict = []
    for i in nounlist: 
        if (i not in removedict) and (i not in stopword_list) and (len(i) > 1):
            cleanlist.append(i) 
    return cleanlist 

def preproc(textlist): 
    nounlist = tokenize(textlist)
    cleanlist = rem_useless(nounlist, stopwords)
    return cleanlist

In [53]:
law_pro_list = preproc(com1_pro['의안명+제안이유'].tolist())


TypeError: No matching overloads found for kr.lucypark.okt.OktInterface.tokenize(int,java.lang.Boolean,java.lang.Boolean), options are:
	public java.util.List kr.lucypark.okt.OktInterface.tokenize(java.lang.String,java.lang.Boolean,java.lang.Boolean)

